# Crawl dữ liệu ở CB-Insights

- Chúng ta Inspect element và nhận thấy dữ liệu nằm ở trong thẻ table của website.
- Tìm hiểu cách crawl dữ liệu table của một website ta tìm được hướng dẫn [ở đây](https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/)
- Làm theo các bước ta crawl được dữ liệu unicorns tính tới 8/2020 

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url = "https://www.cbinsights.com/research-unicorn-companies"
response = requests.get(url)

In [3]:
class HTMLTableParser:
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['class'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [4]:
hp = HTMLTableParser()
table = hp.parse_url(url)[0][1] # Grabbing the table from the tuple
table.head()

,Company,Valuation ($B),Date Joined,Country,Industry,Select Investors
0,Toutiao (Bytedance),$140,4/7/2017,China,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,Didi Chuxing\n\t,$56,12/31/2014,China,Auto & transportation,"Matrix Partners, Tiger\n\t Global Management,..."
2,SpaceX,$46,12/1/2012,United States,Other,"Founders Fund, Draper\n\t Fisher Jurvetson, R..."
3,Stripe,$36,1/23/2014,United States,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Palantir Technologies,$20,5/5/2011,United States,Data management & analytics,"RRE Ventures, Founders\n\t Fund, In-Q-Tel"


## Tiền xử lý dữ liệu sau khi crawl

**Nhận xét**:

- Ta thấy dữ liệu sau khi crawl còn chứa các ký tự '\n\t'.
- Ta sẽ xóa các ký tự này trong bảng.

In [5]:
table.shape

(472, 6)

In [6]:
table = table.replace('\n','', regex=True)
table = table.replace('\t','', regex=True)

In [7]:
table.head()

,Company,Valuation ($B),Date Joined,Country,Industry,Select Investors
0,Toutiao (Bytedance),$140,4/7/2017,China,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,Didi Chuxing,$56,12/31/2014,China,Auto & transportation,"Matrix Partners, Tiger Global Management, Sof..."
2,SpaceX,$46,12/1/2012,United States,Other,"Founders Fund, Draper Fisher Jurvetson, Rothe..."
3,Stripe,$36,1/23/2014,United States,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Palantir Technologies,$20,5/5/2011,United States,Data management & analytics,"RRE Ventures, Founders Fund, In-Q-Tel"


**Nhận xét**:

- Quan sát dữ liệu có vẻ đã hoàn chỉnh, ta xuất ra file CSV
- Bước tiền xử lý dữ liệu để các cột có được kiểu dữ liệu chuẩn ta sẽ làm ở trong file đồ án 1712689-HuynhNgocQuan.ipynb

In [8]:
table.to_csv("CB-Insights_Global-Unicorns_2020.csv")